<a href="https://colab.research.google.com/github/TechTinkerKetki/Induction/blob/main/cynaptics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd


In [ ]:
ls

induction-task.zip  sample_data/


In [ ]:
import zipfile
zip_path = 'induction-task.zip'
# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("induction-task")
    print("extraction completed")



extraction completed


In [ ]:

#Path setup
train='induction-task/Data/Train'
test='induction-task/Data/Test'

In [ ]:
#Preprocessing the data
datagen= ImageDataGenerator(rescale= 1./255, validation_split=0.2)
train_generator = datagen.flow_from_directory(
    train,
    target_size=(128, 128),
    batch_size=15,
    class_mode='binary',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    train,
    target_size=(128, 128),
    batch_size=15,
    class_mode='binary',
    subset='validation'
)



Found 641 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,392,449 (28.20 MB)

 Trainable params: 7,392,449 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#training the model and saving the weights
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    'best_weights.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples,
    validation_steps=validation_generator.samples,
    callbacks=[checkpoint]
)



Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 43/641 ━━━━━━━━━━━━━━━━━━━━ 4:03 407ms/step - accuracy: 0.7106 - loss: 0.8922

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_loss improved from inf to 0.10561, saving model to best_weights.keras
641/641 ━━━━━━━━━━━━━━━━━━━━ 26s 36ms/step - accuracy: 0.8336 - loss: 0.4993 - val_accuracy: 0.9625 - val_loss: 0.1056
Epoch 2/10
 43/641 ━━━━━━━━━━━━━━━━━━━━ 4:02 405ms/step - accuracy: 0.9746 - loss: 0.0792
Epoch 2: val_loss did not improve from 0.10561
641/641 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.9808 - loss: 0.0642 - val_accuracy: 0.9250 - val_loss: 0.1579
Epoch 3/10
 43/641 ━━━━━━━━━━━━━━━━━━━━ 4:19 434ms/step - accuracy: 0.9708 - loss: 0.0887
Epoch 3: val_loss improved from 0.10561 to 0.03534, saving model to best_weights.keras
641/641 ━━━━━━━━━━━━━━━━━━━━ 22s 32ms/step - accuracy: 0.9733 - loss: 0.0749 - val_accuracy: 0.9812 - val_loss: 0.0353
Epoch 4/10
 43/641 ━━━━━━━━━━━━━━━━━━━━ 4:00 403ms/step - accuracy: 0.9968 - loss: 0.0225
Epoch 4: val_loss did not improve from 0.03534
641/641 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - accuracy: 0.9954 - loss: 0.0199 - val_accuracy: 0.9438 - val_lo

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

predictions = []
test_images = os.listdir(test)

for image_name in test_images:
    try:
        image_path = os.path.join(test, image_name)
        image = load_img(image_path, target_size=(128, 128))
        image_array = img_to_array(image) / 255.0
        image_array = np.expand_dims(image_array, axis=0)

        prediction = model.predict(image_array)
        label = 'AI' if prediction[0] < 0.5 else 'Real'
        predictions.append((image_name, label))
    except Exception as e:
        print("Error {image_name}:{e}")
        continue

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Error {image_name}:{e}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [ ]:
#Submission

# DataFrame
submission_df = pd.DataFrame(predictions, columns=['Id', 'Label'])

# Save to CSV
submission_path = 'submission12.csv'
submission_df.to_csv(submission_path, index=False)

print("saved")


saved


In [ ]:
#saving the model
model.save('best_model.keras')
from tensorflow.keras.models import load_model
model=load_model('best_weights.keras')